---
# LEAFToolbox - SL2P 

This notebook contains code blocks to run SL2P based on three different inputs, which are as follows:
1. using SL2p 20m net and 10m net for processing NEON simulated data-output 10m result
2. using SL2p 20m net for processing NEON simulated data-input 20m, output 20m result1
3. using SL2p 20m net and 10m net for processing DenS2 data-input 10m, output 10m result
4. using SL2p 20m net and 10m net for processing S2, 20m net output -20m, 10m net output -10m
5. ALR estimate (input 10m band, output 10m, no nueral network net used in this procedure) 
6. Accuracy analysis
---

In [22]:
import ee
import time
import math
import csv
import json
import os
import numpy as np
import pandas as pd
import folium ; from folium import plugins
import matplotlib.pyplot as plt
import scipy ; from scipy import stats
import scipy.io as sio
import sklearn as skl ; from sklearn import linear_model ; from sklearn import preprocessing
import tensorflow as tf
import pickle
from collections import OrderedDict
from sklearn.metrics import mean_squared_error 
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import r2_score
# import custom modules (files must be in same directory as this notebook)
import feature_collections as fc
import image_bands as ib
import wrapper_nets as wn
import ee_functions as ee_func
import ALR_functions as alr

In [23]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AVHEtk6rug1pdmCN46f-VjBWeARcm6X5yJqq-n2Ky389MFFXOJCqH1hCJJk



Successfully saved authorization token.


---
# Prelim: Define dictionaries

In [219]:
# -----------------------
# SELECT INPUT PARAMETERS
# -----------------------

# variable name
# one of: 'fAPAR', 'fCOVER', 'LAI'
outputName = 'LAI'

# site selection
# one of: 'Geraldton', 'FoxCreek', 'Kouchibouguac', 'Ottawa',
#         'Wabush', 'QueenCharlotte', 'Attawapiskat', 'Eastmain', 'Charlottetown', 'RedBay', 'EaglePlain', 'Kitchener'
#siteSelect = 'Charlottetown'
# siteSelect = 'ABBY'
# siteSelect = 'SJER'
# siteSelect = 'HOPB'
# siteSelect = 'JORN'
# siteSelect = 'SERC'
# siteSelect = 'LENO'
# siteSelect = 'UNDE'
# siteSelect = 'STEI'
siteSelect = 'MCDI'
# siteSelect = 'NOGP'
assetfolder='users/GangHong2/NEON_FILT'

In [220]:
# -----------------------------------------------------
# set parameters based on user-defined parameters above
# -----------------------------------------------------
outputParams = {
    'fAPAR': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'fCOVER': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'LAI': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 8
    }
}

outputScale = outputParams[outputName]['outputScale']
outputOffset = outputParams[outputName]['outputOffset']
outputMax = outputParams[outputName]['outputMax']
responseBand = 'estimate'+outputName

In [221]:
siteParams = {
   
    # 'ABBY': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'HOPB': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'SERC': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'STEI': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'UNDE': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'MCDI': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'LENO': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },  \
    # 'NOGP': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'JORN': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'WREF': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_WREF_DP1_20190712_173947_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_WREF_DP1_20190712_173947_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # },
    # 'SJER': {
    #     'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_10m"),
    #     'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_10m").geometry()
    #     # 'mapCenter': [-80.5, 43.7]
    # }
    
    'ABBY': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'HOPB': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'SERC': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'STEI': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'UNDE': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'MCDI': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'LENO': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },  \
    'NOGP': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'JORN': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'WREF': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_WREF_DP1_20190712_173947_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D16_WREF_DP1_20190712_173947_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'SJER': {
        'testImage': ee.Image("users/GangHong2/NEON_FILT/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_FILT/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    }
}
# all other sites, just follow the site 'ABBY' to add new sites
# users/GangHong2/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m
# users/GangHong2/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m
# users/GangHong2/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m
# users/GangHong2/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m
# users/GangHong2/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m
# users/GangHong2/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m
# users/GangHong2/NEON_D09_NOGP_DP1_20190722_170025_reflectance_10m
# users/GangHong2/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m
# users/GangHong2/NEON_D16_WREF_DP1_20190712_173947_reflectance_10m
# users/GangHong2/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m


mapBounds = siteParams[siteSelect]['mapBounds']
# mapCenter = siteParams[siteSelect]['mapCenter']
testImage = siteParams[siteSelect]['testImage']

# other filters
# maxCloudcover = 10

# export parameters
exportFolder = siteSelect+'_'+outputName
exportDataType = 'int'
exportScale = 20

In [222]:
COLLECTION_OPTIONS = {
    # Sentinel 2 using 20 m bands:
    'COPERNICUS/S2_SR': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'COPERNICUS/S2_SR_10m': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'NEON_Sim_10m': {
      "name": 'NEON_Sim_10m',
      "description": 'NEON Simulated',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
     'NEON_Sim': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
}

VIS_OPTIONS = {
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],   
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
          "NEON_Sim_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "NEON_Sim": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    }
}

In [223]:
### for NEON bands
# def  renameBands(image):
#     bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14','b15','b16']
#     new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
#     return image.select(bands).rename(new_bands)
### for NEON sim filtered bands (excluding Bands 9, 10 of S2 )
def  renameBands(image):
    bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14']
    new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
    return image.select(bands).rename(new_bands)

---
## 1 – SL2P/SL2P10- processing NEON Simulated-independent step

## SL2P (input bands 10m, output band 10m, use 20m neural network coefficents) -Reference

In [224]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [225]:
# filter collection and add ancillary bands

# input_collection = ee.ImageCollection(testImage) \
#                      .map(lambda image: ib.addDate(image)) \
#                      .map(lambda image: image.clip(mapBounds)) \
#                      .map(lambda image: ib.s2MaskClear(image)) \
#                      .map(lambda image: ib.s2MaskLand(image)) \
#                      .map(lambda image: ib.addS2Geometry(colOptions, image))
input_collection = ee.ImageCollection(testImage).map(renameBands)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name +"_"+siteSelect +"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name[26:]+"_"+outputName+"_"+"20m_net" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[16:], 16 here means the lengh of users/GangHong2/NEON_FILT/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [226]:
# print (image_output_names)

In [227]:
# export tasks to Earth Engine
assetfolder='users/GangHong2/NEON_FILT_RESULT'
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,
                                               # image_names=[siteSelect+'_'+outputName+'_SL2P'],
                                               image_names = image_output_names,
                                               scale=10,
                                               # asset_folder='users/kateharvey/SL2P_images',
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## SL2P10 (input bands 10m, output 10m,  10m neural network coefficients) -solution B

In [ ]:
# parse the networks
colName ='NEON_Sim_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [ ]:
input_collection_10m = ee.ImageCollection(testImage).map(renameBands)

In [ ]:
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

In [ ]:
# print (scaled_input_collection_10m.first().getInfo())

In [ ]:
# performs same procedure as above, using SL2P10 network
# applies algorithm to 10 m bands ; generates a 10 m map

# filter collection and add ancillary bands
# input_collection_10m = ee.ImageCollection(testImage) \
#                      .map(lambda image: ib.addDate(image)) \
#                      .map(lambda image: image.clip(mapBounds)) \
#                      .map(lambda image: ib.s2MaskClear(image)) \
#                      .map(lambda image: ib.s2MaskLand(image)) \
#                      .map(lambda image: ib.addS2Geometry(colOptions, image))
# input_collection_10m = ee.ImageCollection(testImage)

# get partition used to select network
# partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')
# partition = (COLLECTION_OPTIONS[colName]["partition"]).rename('partition')

# # pre process input imagery and flag invalid inputs
# scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
#                                                   .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))




# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name[16:]+"_"+outputName+"_"+"10m_net" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()])

ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [ ]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,
                                                   # image_names=[siteSelect+'_'+outputName+'_SL2P10'],
                                                   image_names = image_output_names_10m,
                                                   scale=10,
                                                   # asset_folder='users/kateharvey/SL2P10_images',
                                                   asset_folder=assetfolder,
                                                   # data_type=exportDataType,
                                                   data_type='float',
                                                   max_pixels=1e13)

## 2. SL2P for processing 20m NEON -independent step

## resample 10m NEON to 20m

In [41]:
inputcollection_10m = ee.ImageCollection(testImage).map(renameBands) ## rename testImage bands based on the bands name for input SL2P
proj_10m=inputcollection_10m.first().select('B1').projection().getInfo()  ## get the projection of one band
inputImage_20m = inputcollection_10m.first().resample('bilinear').reproject(crs=proj_10m['crs'], scale=20) ## resample the image to 20m

## SL2P (input band 20m, ouput band 20m, 20m net) -NULL Hypothesis

In [108]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [109]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(inputImage_20m)##.map(renameBands)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name +"_"+siteSelect +"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name[16:61]+"_20m_"+outputName+"_"+"20m_net" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[16:], 16 here means the lengh of users/GangHong2/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [110]:
# export tasks to Earth Engine
export_task_20m = ee_func.export_collection_to_gee(collection=export_collection,
                                                   num_images=1,                                                   
                                                   image_names = image_output_names,
                                                   scale=10,   ## inputband 20m, the acutal output is 20m, but here 10m for comparison with others                                            
                                                   asset_folder=assetfolder,                                                   
                                                   data_type='float',
                                                   max_pixels=1e13)

## 3– SL2P/SL2P10- Processing DenS2 -independent step

In [31]:
### make an imagecollection from DenS2 imageries- 10 totally
img1=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190822T173909_N0213_R098_T13SCS_20190822T220238'))
img2=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190826T153819_N0213_R011_T18TYN_20190826T195901'))
img3=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210422T162829_N0300_R083_T16SCA_20210422T204151'))
img4=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'))
img5=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190608T164849_N0212_R026_T15TYL_20190608T212115'))
img6=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210401T183919_N0300_R070_T11SKB_20210401T224235'))
img7=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20200627T173909_N0214_R098_T14TLS_20200627T213600'))
img8=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210808T154809_N0301_R054_T18SUJ_20210808T201351'))
img9=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2A_MSIL2A_20200713T170901_N0214_R112_T14SQJ_20200713T213310'))
img10=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2A_MSIL2A_20190606T165901_N0212_R069_T16TCS_20190606T212006'))

## an image collection
DenS2_Col=img1.merge(img2).merge(img3).merge(img4).merge(img5).merge(img6).merge(img7).merge(img8).merge(img9).merge(img10)


## preparing image for runing SL2P

In [217]:
### get the asset ID, like 'users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
neon_id=testImage.get("system:id").getInfo()
neon_name=neon_id[16:]   ## slice the neon_id to get the name, like 'NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
## based on the property name 'NEON' to find denS2
denS2=DenS2_Col.filter(ee.Filter.eq('NEON', neon_name))
## get the DenS2 name, like 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'
denS2_Name=denS2.first().get("system:id").getInfo()[22:]
## find S2 from GEE from denS2 file name
S2img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(denS2.geometry()).filter(ee.Filter.eq('PRODUCT_ID', denS2_Name))
### select S2 based on DenS2 image geometry and keep band 'SCL' for masking Land
S2_selected=S2img.first().clip(denS2.geometry()).select('SCL')
## merge S2 SCL band and DenS2 bands
newS2=S2_selected.addBands(denS2.first())
## rename bands 
inputImage_bands = ee.List(['SCL','B4', 'B3', 'B2', 'B8','B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
inputImage = newS2.rename(inputImage_bands)
# print(inputImage.bandNames().getInfo())

## SL2P (input band 10m, ouput band 10m, 20m net) -Solution A

In [10]:
# parse the networks
colName = 'COPERNICUS/S2_SR'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [11]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(inputImage).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name+"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name+"_"+siteSelect+"_"+outputName+"_DSen2_20m_net3" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [12]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                             
                                               image_names = image_output_names,
                                               scale=10,                                          
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## SL2P10 (inputband 10m, output band 10m, 10m net)

In [ ]:
# parse the networks
colName = 'COPERNICUS/S2_SR_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [ ]:
# performs same procedure as above, using SL2P10 network
# applies algorithm to 10 m bands ; generates a 10 m map

# filter collection and add ancillary bands
input_collection_10m = ee.ImageCollection(inputImage).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.s2MaskLand(image)) \
                                                  .map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_DenS2_10m_net" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [ ]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,                                                  
                                                   image_names = image_output_names_10m,
                                                   scale=10,                                                  
                                                   asset_folder=assetfolder,                                                
                                                   data_type='float',
                                                   max_pixels=1e13)

## SL2P (input normalized band 10m, ouput band 10m, 20m net) 

In [140]:
assetfolder='users/GangHong2/DSen2_norm'
# LR_COE=ee.FeatureCollection('users/GangHong2/LinearRegression_COE')
LR_COE=ee.FeatureCollection('users/GangHong2/LinearRegression_COE2')
# sitename='ABBY'
feat=LR_COE.filter(ee.Filter.equals('site',siteSelect))  ## 'site' is the property name, a varible, site is actual site name, 
B2_slope=feat.first().get('B2_slope').getInfo()
B2_intercept=feat.first().get('B2_intercept').getInfo()
B3_slope=feat.first().get('B3_slope').getInfo()
B3_intercept=feat.first().get('B3_intercept').getInfo()
B4_slope=feat.first().get('B4_slope').getInfo()
B4_intercept=feat.first().get('B4_intercept').getInfo()
B5_slope=feat.first().get('B5_slope').getInfo()
B5_intercept=feat.first().get('B5_intercept').getInfo()
B6_slope=feat.first().get('B6_slope').getInfo()
B6_intercept=feat.first().get('B6_intercept').getInfo()
B7_slope=feat.first().get('B7_slope').getInfo()
B7_intercept=feat.first().get('B7_intercept').getInfo()
B8_slope=feat.first().get('B8_slope').getInfo()
B8_intercept=feat.first().get('B8_intercept').getInfo()
B8A_slope=feat.first().get('B8A_slope').getInfo()
B8A_intercept=feat.first().get('B8A_intercept').getInfo()
B11_slope=feat.first().get('B11_slope').getInfo()
B11_intercept=feat.first().get('B11_intercept').getInfo()
B12_slope=feat.first().get('B12_slope').getInfo()
B12_intercept=feat.first().get('B12_intercept').getInfo()

print (B2_slope)


0.666444204


In [141]:
### get the asset ID, like 'users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
neon_id=testImage.get("system:id").getInfo()
neon_name=neon_id[16:]   ## slice the neon_id to get the name, like 'NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
## based on the property name 'NEON' to find denS2
denS2=DenS2_Col.filter(ee.Filter.eq('NEON', neon_name))
## get the DenS2 name, like 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'
denS2_Name=denS2.first().get("system:id").getInfo()[22:]
## find S2 from GEE from denS2 file name
S2img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(denS2.geometry()).filter(ee.Filter.eq('PRODUCT_ID', denS2_Name))
### select S2 based on DenS2 image geometry and keep band 'SCL' for masking Land
S2_selected=S2img.first().clip(denS2.geometry()).select('SCL')
## merge S2 SCL band and DenS2 bands
newS2=S2_selected.addBands(denS2.first())
## rename bands 
inputImage_bands = ee.List(['SCL','B4', 'B3', 'B2', 'B8','B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
inputImg = newS2.rename(inputImage_bands)
# print(inputImage.bandNames().getInfo())

In [142]:
B2=inputImg.select('B2').multiply(B2_slope).add(B2_intercept)
B3=inputImg.select('B3').multiply(B3_slope).add(B3_intercept)
B4=inputImg.select('B4').multiply(B4_slope).add(B4_intercept)
B5=inputImg.select('B5').multiply(B5_slope).add(B5_intercept)
B6=inputImg.select('B6').multiply(B6_slope).add(B6_intercept)
B7=inputImg.select('B7').multiply(B7_slope).add(B7_intercept)
B8=inputImg.select('B8').multiply(B8_slope).add(B8_intercept)
B8A=inputImg.select('B8A').multiply(B8A_slope).add(B8A_intercept)
B11=inputImg.select('B11').multiply(B11_slope).add(B11_intercept)
B12=inputImg.select('B12').multiply(B12_slope).add(B12_intercept)

In [143]:
inputImage=S2_selected.addBands(B4).addBands(B3).addBands(B2).addBands(B8).addBands(B5).addBands(B6).addBands(B7).addBands(B8A).addBands(B11).addBands(B12)

In [144]:
print (inputImage.bandNames().getInfo())

['SCL', 'B4', 'B3', 'B2', 'B8', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12']


In [145]:
# parse the networks
colName = 'COPERNICUS/S2_SR'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [146]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(inputImage).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name+"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name+"_"+siteSelect+"_"+outputName+"_DSen2_20m_net_norm1" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [147]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                             
                                               image_names = image_output_names,
                                               scale=10,                                          
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## 4 – SL2P/SL2P10- Processing S2 -independent step

In [40]:
### make an imagecollection from DSen2 imageries
img1=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190822T173909_N0213_R098_T13SCS_20190822T220238'))
img2=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190826T153819_N0213_R011_T18TYN_20190826T195901'))
img3=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210422T162829_N0300_R083_T16SCA_20210422T204151'))
img4=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'))
img5=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190608T164849_N0212_R026_T15TYL_20190608T212115'))
img6=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210401T183919_N0300_R070_T11SKB_20210401T224235'))
img7=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20200627T173909_N0214_R098_T14TLS_20200627T213600'))
img8=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210808T154809_N0301_R054_T18SUJ_20210808T201351'))
img9=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2A_MSIL2A_20200713T170901_N0214_R112_T14SQJ_20200713T213310'))
img10=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2A_MSIL2A_20190606T165901_N0212_R069_T16TCS_20190606T212006'))

## an image collection
DenS2_Col=img1.merge(img2).merge(img3).merge(img4).merge(img5).merge(img6).merge(img7).merge(img8).merge(img9).merge(img10)


In [87]:
### get the asset ID, like 'users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
neon_id=testImage.get("system:id").getInfo()
neon_name=neon_id[16:]   ## slice the neon_id to get the name, like 'NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
## based on the property name 'NEON' to find denS2
denS2=DenS2_Col.filter(ee.Filter.eq('NEON', neon_name))
## get the DenS2 name, like 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'
denS2_Name=denS2.first().get("system:id").getInfo()[22:]
## find S2 from GEE from denS2 file name
S2img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(denS2.geometry()).filter(ee.Filter.eq('PRODUCT_ID', denS2_Name))
### select S2 based on DenS2 image geometry and keep band 'SCL' for masking Land
S2_selected=S2img.first().clip(denS2.geometry()).select(['B2', 'B3', 'B4', 'B5','B6', 'B7', 'B8', 'B8A', 'B11', 'B12','SCL'])

## SL2P (input bands 20m, output 20m,  20m net)

In [88]:
# parse the networks
colName = 'COPERNICUS/S2_SR'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [89]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(S2_selected).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name+"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name+"_"+siteSelect+"_"+outputName+"_S2_20m_net3" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [90]:
print (scaled_input_collection.first().bandNames().getInfo())

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'SCL', 'cosVZA', 'cosSZA', 'cosRAA', 'QC']


In [26]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                             
                                               image_names = image_output_names,
                                               scale=20,                ## output 20m                
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## SL2P10 (input bands 10m, output 10m, 10m net)

In [27]:
# parse the networks
colName = 'COPERNICUS/S2_SR_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [28]:
# performs same procedure as above, using SL2P10 network
# applies algorithm to 10 m bands ; generates a 10 m map

# filter collection and add ancillary bands
input_collection_10m = ee.ImageCollection(S2_selected).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.s2MaskLand(image)) \
                                                  .map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_S2_10m_net" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [29]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,                                                  
                                                   image_names = image_output_names_10m,
                                                   scale=10,                                                  
                                                   asset_folder=assetfolder,                                                
                                                   data_type='float',
                                                   max_pixels=1e13)

## SL2P (input bands normalized 20m, output 10m to keep 10m bands,  20m net)

In [133]:
assetfolder='users/GangHong2/S2_norm'
LR_COE=ee.FeatureCollection('users/GangHong2/LinearRegression_COE2')
# sitename='ABBY'
feat=LR_COE.filter(ee.Filter.eq('site',siteSelect))  ## 'site' is the property name, a varible, site is actual site name, 
B2_slope=feat.first().get('B2_slope').getInfo()
B2_intercept=feat.first().get('B2_intercept').getInfo()
B3_slope=feat.first().get('B3_slope').getInfo()
B3_intercept=feat.first().get('B3_intercept').getInfo()
B4_slope=feat.first().get('B4_slope').getInfo()
B4_intercept=feat.first().get('B4_intercept').getInfo()
B5_slope=feat.first().get('B5_slope').getInfo()
B5_intercept=feat.first().get('B5_intercept').getInfo()
B6_slope=feat.first().get('B6_slope').getInfo()
B6_intercept=feat.first().get('B6_intercept').getInfo()
B7_slope=feat.first().get('B7_slope').getInfo()
B7_intercept=feat.first().get('B7_intercept').getInfo()
B8_slope=feat.first().get('B8_slope').getInfo()
B8_intercept=feat.first().get('B8_intercept').getInfo()
B8A_slope=feat.first().get('B8A_slope').getInfo()
B8A_intercept=feat.first().get('B8A_intercept').getInfo()
B11_slope=feat.first().get('B11_slope').getInfo()
B11_intercept=feat.first().get('B11_intercept').getInfo()
B12_slope=feat.first().get('B12_slope').getInfo()
B12_intercept=feat.first().get('B12_intercept').getInfo()
print (B2_slope)

0.666444204


In [134]:
### make an imagecollection from DSen2 imageries
img1=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190822T173909_N0213_R098_T13SCS_20190822T220238'))
img2=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190826T153819_N0213_R011_T18TYN_20190826T195901'))
img3=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210422T162829_N0300_R083_T16SCA_20210422T204151'))
img4=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'))
img5=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20190608T164849_N0212_R026_T15TYL_20190608T212115'))
img6=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210401T183919_N0300_R070_T11SKB_20210401T224235'))
img7=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20200627T173909_N0214_R098_T14TLS_20200627T213600'))
img8=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2B_MSIL2A_20210808T154809_N0301_R054_T18SUJ_20210808T201351'))
img9=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2A_MSIL2A_20200713T170901_N0214_R112_T14SQJ_20200713T213310'))
img10=ee.ImageCollection(ee.Image('users/GangHong2/DSen2/S2A_MSIL2A_20190606T165901_N0212_R069_T16TCS_20190606T212006'))

## an image collection
DenS2_Col=img1.merge(img2).merge(img3).merge(img4).merge(img5).merge(img6).merge(img7).merge(img8).merge(img9).merge(img10)

In [135]:
### get the asset ID, like 'users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
neon_id=testImage.get("system:id").getInfo()
neon_name=neon_id[16:]   ## slice the neon_id to get the name, like 'NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
## based on the property name 'NEON' to find denS2
denS2=DenS2_Col.filter(ee.Filter.eq('NEON', neon_name))
## get the DenS2 name, like 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'
denS2_Name=denS2.first().get("system:id").getInfo()[22:]
## find S2 from GEE from denS2 file name
S2img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(denS2.geometry()).filter(ee.Filter.eq('PRODUCT_ID', denS2_Name))
### select S2 based on DenS2 image geometry and keep band 'SCL' for masking Land
inputImg=S2img.first().clip(denS2.geometry()).select(['B2', 'B3', 'B4', 'B5','B6', 'B7', 'B8', 'B8A', 'B11', 'B12','SCL'])

In [136]:
B2=inputImg.select('B2').multiply(B2_slope).add(B2_intercept)
B3=inputImg.select('B3').multiply(B3_slope).add(B3_intercept)
B4=inputImg.select('B4').multiply(B4_slope).add(B4_intercept)
B5=inputImg.select('B5').multiply(B5_slope).add(B5_intercept)
B6=inputImg.select('B6').multiply(B6_slope).add(B6_intercept)
B7=inputImg.select('B7').multiply(B7_slope).add(B7_intercept)
B8=inputImg.select('B8').multiply(B8_slope).add(B8_intercept)
B8A=inputImg.select('B8A').multiply(B8A_slope).add(B8A_intercept)
B11=inputImg.select('B11').multiply(B11_slope).add(B11_intercept)
B12=inputImg.select('B12').multiply(B12_slope).add(B12_intercept)

In [137]:
S2_selected=inputImg.select('SCL').addBands(B2).addBands(B3).addBands(B4).addBands(B5).addBands(B6).addBands(B7).addBands(B8).addBands(B8A).addBands(B11).addBands(B12)

In [138]:
# print (S2_selected.bandNames().getInfo())

In [139]:
# parse the networks
colName = 'COPERNICUS/S2_SR'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [140]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(S2_selected).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name+"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name+"_"+siteSelect+"_"+outputName+"_S2_20m_net_norm1" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [141]:
# print (scaled_input_collection.first().bandNames().getInfo())

In [143]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                             
                                               image_names = image_output_names,
                                               scale=10,                ## output 10m                
                                               asset_folder=assetfolder,
                                               data_type='float',
                                               max_pixels=1e13)

## 5. ALR estimate (input 10m band, output 10m, no nueral network net but random forest used in this procedure)-independent step Solution C

In [92]:
## ALR function for estimation,after feature selection, a simple random forest estiamte applied
def func_ALR(temp_image, responsedBand, outputName, mapBounds):
   
    # inputImage = ee.Image(temp_image).select(1,2,3,7,22,23,27,28,29,30,31,32)
    # inputImage = ee.Image(temp_image).select(1,2,3,7,16,17,18,19,20,21)
    # inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'QA60', 'date', 'estimate'+outputName, 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'estimate'+outputName, 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    # inputImage = inputImage.rename(inputImage_bands)
    inputImage = ee.Image(temp_image).select(inputImage_bands)
    input_VI_definition = ee.List([
                                   "GI      = b('B3')/b('B4')",                               
                                   "SGI     = b('B8')/b('B4')",                                
                                   "GVI     = (b('B8')/b('B3'))-1",                             
                                   "NDVI3   = ((b('B8')-b('B4'))/(b('B8')))+b('B4')",                                
                                   "NDVI    = (b('B8')-b('B4'))/(b('B8')+b('B4'))",
                                   "GNDVI   = (b('B8')-b('B3'))/(b('B8')+b('B3'))",                                
                                   "NDGI    = (b('B3')-b('B4'))/(b('B3')+b('B4'))",                                 
                                   "EVI     = 2.5*((b('B8')-b('B4'))/(b('B8')+6*b('B4')-7.5*b('B3')+1))",
                                   "EVI2    = 2.5*((b('B8')-b('B4'))/(b('B8')+2.4*b('B4')+1))",
                                   "RDVI    = (b('B8')-b('B4'))/((b('B8')+b('B4'))**0.5)",
                                   "MSR     = ((b('B8')/b('B4'))-1)/((b('B8')/b('B4'))**0.5+1)",                            
                                   "MSAVI2  = 0.5*(2*b('B8')+1-((2*b('B8')+1)**2-8*(b('B8')-b('B4')))**0.5)",                                
                                   "NLI     = ((b('B8')**2)-b('B4'))/((b('B8')**2)+b('B4'))"])

    # names of bands to pass to ALR method (excluding metadata and other non-spectral bands)
    input_bandNames = ['B2', 'B3', 'B4', 'B8', 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI']
    
    def format_image(image, image_bands, response_band, VI_definition):
        image = ee.Image(image)
        image_bands = ee.List(image_bands)
        response_band = ee.String(response_band)
        VI_definition = ee.List(VI_definition)
        
        # image_bands specifices a list of the names of the bands used in defining the expressions for VIs in VI_definition
        image = image.rename(image_bands).toDouble()
        
        # Generate an ImageCollection from a list of expressions defining a set of VIs using the bands available in the image
        VIimageCollection = ee.ImageCollection(VI_definition.map(lambda expr: image.expression(expr)))
        VIimage = VIimageCollection.toBands().regexpRename("[0-9]+_", "")
        
        # Reorder the bands in the image so the response band is the first band in the image
        feature_bands = image_bands.remove(response_band)
        
        return ee.Image(image.select(response_band).addBands(VIimage).addBands(image.select(feature_bands)))

    
    inputImage = format_image(inputImage, inputImage_bands, responseBand, input_VI_definition)
    
    def scale_image(image, response_band):
        image = ee.Image(image)
        response_band = ee.String(response_band)
        
        def get_num_pixels(image):
    
            # get image height
            def get_height(image):
                height = image.getInfo()["bands"][0]["dimensions"][0]
                return height

            # get image width
            def get_width(image):
                width = image.getInfo()["bands"][0]["dimensions"][1]
                return width

            image_height = get_height(image)
            image_width = get_width(image)
            image_pixels = image_height*image_width

            return image_pixels
        
        image_pixels = ee.Number(get_num_pixels(image))
        
        # Set up lists containing the input/feature bands in the image
        bandList = image.bandNames()
        featureList = bandList.remove(response_band)
        num_bands = bandList.length()
        num_features = featureList.length()
        
        # We will be using the reduceRegion() function on images from Earth Engine, 
        # which will process up to a specified number of pixels from the image to generate the outputs of the reducer
        max_pixels = image_pixels.min(10000000)
        # best_effort = ee.Algorithms.If(image_pixels.gt(max_pixels), True, False)
        
        # Set default projection and scale using the response band
        defaultScale = image.select(response_band).projection().nominalScale()
        defaultCrs = image.select(response_band).projection().crs()
        image = image.setDefaultProjection(crs=defaultCrs, scale=defaultScale)
        
        # Center all of the bands in the image for LARs
        # We will centre the sampled data later as well as reduceRegion() is not precise enough
        meanImage = image.subtract(image.reduceRegion(reducer=ee.Reducer.mean(), \
                                    scale=defaultScale, bestEffort=True, maxPixels=max_pixels).toImage(bandList))
        
        # Separate the image into features (X) and response (y) as we need to standardize the input features
        X = meanImage.select(featureList)
        y = meanImage.select(response_band)
        
        # Standardize the input features
        X = X.divide(X.reduceRegion(reducer=ee.Reducer.stdDev(), bestEffort=True, maxPixels=max_pixels).toImage(featureList))
        
        return X.addBands(y)

    
    scaledImage = scale_image(inputImage, responseBand)
    
    
    def ee_LARS(input_image, input_bandNames, response_bandName, num_nonzero_coefficients, num_samples):
        image = ee.Image(input_image)
        feature_list = ee.List(input_bandNames)
        response_band = ee.String(response_bandName)
        full_band_list = ee.List(feature_list).add(response_band)
        num_nonzero_coefficients = ee.Number(num_nonzero_coefficients)
        num_samples = ee.Number(num_samples)
        def get_num_pixels(image):
    
            # get image height
            def get_height(image):
                height = image.getInfo()["bands"][0]["dimensions"][0]
                return height

            # get image width
            def get_width(image):
                width = image.getInfo()["bands"][0]["dimensions"][1]
                return width

            image_height = get_height(image)
            image_width = get_width(image)
            image_pixels = image_height*image_width

            return image_pixels
        image_pixels = ee.Number(get_num_pixels(image))
        
        # Randomly sample pixels in the image at native resolution into a FeatureCollection
        input_collection = image.sample(numPixels=num_samples.min(image_pixels))
        n = input_collection.size()
        m = feature_list.length()
        
        # Use an aggregate array function over the FeatureCollection and map the function over each feature in the band list
        # to generate a dictionary of all of the samples retrieved
        inputs = ee.Dictionary.fromLists(full_band_list, full_band_list.map(lambda feature: input_collection.aggregate_array(feature)))
        
        # Although we may call our scale_image function on the input image, the reduceRegion() function used to determine the mean
        # and standard deviation of each band in the image over the entire region is not precise enough over a large image
        # so we must recenter all of the bands in the image and now we can also normalize (L2 norm) each input feature as required
        # by the LARs algorithm
        
        # Use an aggregate_mean function over the feature collection to get the mean of each band
        input_means = ee.Dictionary.fromLists(full_band_list, full_band_list.map(lambda feature: input_collection.aggregate_mean(feature)))

        def centre_inputs(key, value):
            key_mean = input_means.getNumber(key)
            return ee.List(value).map(lambda sample: ee.Number(sample).subtract(key_mean))
        
        
        # Center bands by mapping over the list of features and then a subtracting over the list of samples for each band
        inputs = inputs.map(centre_inputs)

        # Separate the response variable samples into its own vector
        y = inputs.toArray([response_band]).reshape([-1,1])

        # Remove response band from the feature collection by selecting only bands in the feature list
        inputs = inputs.select(feature_list)
        
        # Generate a dictionary of all of the L2 norms of the input features using a custom mapped function
        input_norms = inputs.map(lambda key, value: ee.Number(ee.List(value).map(lambda sample: ee.Number(sample).pow(2)).reduce(ee.Reducer.sum())).pow(0.5))

        def norm_inputs(key, value):
            key_norm = input_norms.getNumber(key)
            return ee.List(value).map(lambda sample: ee.Number(sample).divide(key_norm))
        
        # Normalize all of the features by mapping a function over the list of features
        # and then map a division over the list of all of the samples of the feature
        inputs = inputs.map(norm_inputs)
        
        # Generate the array of samples using the dictionary
        X = inputs.toArray(feature_list).transpose()

        # Find the first best predictor of the response to initialize the main LARs loop
        initial_prediction = ee.Array(ee.List.repeat([0], n))
        c = X.transpose().matrixMultiply(y.subtract(initial_prediction))
        c_abs = c.abs()
        C_maxLoc = c_abs.project([0]).argmax()
        add_feature = C_maxLoc.getNumber(0)
        A = ee.List([add_feature])
        
        # Create a dicitionary of initial inputs to pass into the main LARs iterative loop
        # The iterate function in Earth Engine processes each iteration as a tree of iterations with no access to any variables
        # from previous iterations (only those that are passed to the next iteration)
        # so we must pass both the current prediction and the active set of features (with non-zero coefficients), A
        initial_inputs = ee.Dictionary({'prediction': initial_prediction, 'A': A})

        def LARs_regression(iteration, inputs):
            inputs = ee.Dictionary(inputs)

            # Find the active set of features, A (predictors with non-zero coefficients)
            A = ee.List(inputs.get('A'))
            # A_list is an array used to mask the full array of input samples and the correlation vector
            A_list = ee.Array(ee.List.sequence(0, m.subtract(1))\
                              .map(lambda index: A.contains(index)).replaceAll(False, 0).replaceAll(True, 1)).reshape([-1,1])

            # The following matrix algebra determines the next most correlated variable, or the next best predictor considering the
            # current features in the active set, A, as well as the magnitude to adjust the prediction vector to ensure all of the
            # features in the active set are equally correlated to response vector
            prediction = inputs.getArray('prediction')
            c = X.transpose().matrixMultiply(y.subtract(prediction))
            c_abs = c.abs()
            C_max = c_abs.get(c_abs.argmax())
            s_A = c.divide(c_abs).mask(A_list)
            X_A = X.mask(A_list.transpose())
            G_Ai = X_A.transpose().matrixMultiply(X_A).matrixInverse()
            G1 = G_Ai.matrixMultiply(s_A)
            A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
            w_A = G1.multiply(A_A)
            u_A = X_A.matrixMultiply(w_A)
            a = X.transpose().matrixMultiply(u_A)
            a = a.project([0])
            c = c.project([0])

            def compute_gammaArray(index_j):
                minus_j = C_max.subtract(c.get([index_j])).divide(A_A.subtract(a.get([index_j])))
                plus_j = C_max.add(c.get([index_j])).divide(A_A.add(a.get([index_j])))
                return ee.List([minus_j, plus_j]).filter(ee.Filter.gte('item', 0)).reduce(ee.Reducer.min())

            A_c = ee.List.sequence(0, m.subtract(1)).removeAll(A)
            gammaArray = A_c.map(compute_gammaArray)
            gamma = gammaArray.reduce(ee.Reducer.min())
            min_location = gammaArray.indexOf(gamma)
            add_feature = A_c.getNumber(min_location)

            # Update active set of variables with next best predictor from non-active set and update prediction vector
            A = A.add(add_feature)
            prediction = prediction.add(u_A.multiply(gamma))

            return ee.Dictionary({'prediction': prediction, 'A': A})


        # The final iteration of LARs (if selecting all input variables) requires a different method to determine magnitude for
        # adjusting the magnitude of the prediction vector, as the regular LARs iteration relies on variables in non-active set
        # In the final iteration there will be no variables in the non-active set, so the method will not work
        def LARs_final_iteration(iteration, inputs):
            inputs = ee.Dictionary(inputs)
            A = ee.List(inputs.get('A'))

            prediction = inputs.getArray('prediction')
            c = X.transpose().matrixMultiply(y.subtract(prediction))
            c_abs = c.abs()
            C_max = c_abs.get(c_abs.argmax())        

            s_A = c.divide(c_abs)
            G_Ai = X.transpose().matrixMultiply(X).matrixInverse()
            G1 = G_Ai.matrixMultiply(s_A)
            A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
            w_A = G1.multiply(A_A)
            u_A = X.matrixMultiply(w_A)

            gamma = C_max.divide(A_A)
            prediction = prediction.add(u_A.multiply(gamma))

            return ee.Dictionary({'prediction': prediction, 'A': A})

        # Actually carrying out the iterations by iterating over a placeholder list (sequence from 1 to the number of non-zero
        # variables that the user wishes to select as predictors for the response)
        iterations = ee.List.sequence(1, m.subtract(1).min(num_nonzero_coefficients))
        penultimate_outputs = iterations.iterate(LARs_regression, initial_inputs)
        final_outputs = ee.Dictionary(ee.Algorithms.If(num_nonzero_coefficients.gte(m), \
                                LARs_final_iteration(m, penultimate_outputs), penultimate_outputs))
        
        final_prediction = final_outputs.getArray('prediction')

        A = ee.List(final_outputs.get('A'))

        feature_path = A.slice(0, num_nonzero_coefficients).map(lambda index: feature_list.getString(index))        
        return feature_path

    
    select_features = ee_LARS(scaledImage, input_bandNames, responseBand, 5, 50000)
    #unclassified = ee.Image(cloud_folder+'/'+siteSelect+'_'+outputName+'_VI')
    unclassified = ee.Image(inputImage)
    # bands = ee.List([responseBand, 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI',
    #                  'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI', 'B2', 'B3', 'B4', 'B8',
    #                  'QA60', 'date', 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    bands = ee.List([responseBand, 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI',
                     'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI', 'B2', 'B3', 'B4', 'B8',
                     'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    unclassified = unclassified.rename(bands)

    # prediction bands (equivalent to select_features, with responseBand)
    bands = select_features
    input_bands = select_features.add(responseBand)
    training_data = ee.FeatureCollection(unclassified.sample(numPixels=1000, seed=1).select(input_bands))
    # implement regression tree with Random Forest algorithm
    # optional parameters for smileRandomForest(): variablesPerSplit, minLeafPopulation, bagFraction, maxNodes, seed
    rf_classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(features=training_data,
                                                                                           classProperty=responseBand,
                                                                                           inputProperties=input_bands)
    rf_classified = unclassified.select(bands).classify(rf_classifier, 'ALR_'+responseBand).clip(mapBounds)
    return temp_image.addBands(rf_classified)
    

In [102]:
## use a biophysical parameter result as an input
assetname='users/GangHong2/NEON/20210719T190919_20210719T191700_T10TER_ABBY_LAI_DSen2_20m_net'
img=ee.Image(assetname)
mapBounds=img.geometry()
output_Name=assetname[21:]+'_ALR' ### index number neeed to be adjusted to get the meaningful file name '20210719T190919_20210719T191700_T10TER_ABBY_LAI_DenS2_20m_net_ALR'

In [ ]:
outputName = 'LAI'
responseBand = 'estimate'+outputName
ALR_result=func_ALR(img, responseBand, outputName, mapBounds)
# print (ALR_result.bandNames().getInfo())

In [106]:
# export formatted image to GEE asset 
assetfolder='users/GangHong2/ALR'
export = ee.ImageCollection(ALR_result)
export_task = ee_func.export_collection_to_gee(collection=export,
                                                 num_images=1,                                                                                              
                                                 # image_names=[siteSelect+'_'+outputName+'_ALR'],
                                                 image_names=[output_Name],
                                                 asset_folder=assetfolder,                                                 
                                                 scale=10,
                                                 data_type='float',
                                                 max_pixels=1e13)

## normalize S2 first, then run ALR

## 6. Accuracy analysis
    -RMSE
    -SSIM
    -R2

In [8]:
# ## make a dictionry list for reference image and the image before normalization to be compared based on site 
input_list=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20210719T190919_20210719T191700_T10TER_ABBY_LAI_DSen2_20m_net'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20210401T183919_20210401T184709_T11SKB_SJER_LAI_DSen2_20m_net'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20190822T173909_20190822T175453_T13SCS_JORN_LAI_DSen2_20m_net'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20200627T173909_20200627T174744_T14TLS_NOGP_LAI_DSen2_20m_net'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20210422T162829_20210422T163638_T16SCA_LENO_LAI_DSen2_20m_net'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20200713T170901_20200713T171937_T14SQJ_MCDI_LAI_DSen2_20m_net'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20190606T165901_20190606T170333_T16TCS_UNDE_LAI_DSen2_20m_net'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20190608T164849_20190608T165019_T15TYL_STEI_LAI_DSen2_20m_net'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20210808T154809_20210808T155521_T18SUJ_SERC_LAI_DSen2_20m_net'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_result/20190826T153819_20190826T154455_T18TYN_HOPB_LAI_DSen2_20m_net'}]

In [9]:
## make a dictionry list for reference image and the image to be compared based on site 
# input_list0=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON/20210719T190919_20210719T191700_T10TER_ABBY_LAI_DSen2_20m_net_norm'}
# ]

In [10]:
## make a dictionry list for reference image and the image (normed DSen2) to be compared based on site 
input_list2=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210719T190919_20210719T191700_T10TER_ABBY_LAI_DSen2_20m_net_norm0'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210401T183919_20210401T184709_T11SKB_SJER_LAI_DSen2_20m_net_norm0'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190822T173909_20190822T175453_T13SCS_JORN_LAI_DSen2_20m_net_norm0'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20200627T173909_20200627T174744_T14TLS_NOGP_LAI_DSen2_20m_net_norm0'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210422T162829_20210422T163638_T16SCA_LENO_LAI_DSen2_20m_net_norm0'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20200713T170901_20200713T171937_T14SQJ_MCDI_LAI_DSen2_20m_net_norm0'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190606T165901_20190606T170333_T16TCS_UNDE_LAI_DSen2_20m_net_norm0'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190608T164849_20190608T165019_T15TYL_STEI_LAI_DSen2_20m_net_norm0'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210808T154809_20210808T155521_T18SUJ_SERC_LAI_DSen2_20m_net_norm0'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190826T153819_20190826T154455_T18TYN_HOPB_LAI_DSen2_20m_net_norm0'}]

In [11]:
## make a dictionry list for reference image and the image (normed S2)to be compared based on site 
input_list3=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210719T190919_20210719T191700_T10TER_ABBY_LAI_S2_20m_net_norm0'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210401T183919_20210401T184709_T11SKB_SJER_LAI_S2_20m_net_norm0'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190822T173909_20190822T175453_T13SCS_JORN_LAI_S2_20m_net_norm0'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20200627T173909_20200627T174744_T14TLS_NOGP_LAI_S2_20m_net_norm0'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210422T162829_20210422T163638_T16SCA_LENO_LAI_S2_20m_net_norm0'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20200713T170901_20200713T171937_T14SQJ_MCDI_LAI_S2_20m_net_norm0'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190606T165901_20190606T170333_T16TCS_UNDE_LAI_S2_20m_net_norm0'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190608T164849_20190608T165019_T15TYL_STEI_LAI_S2_20m_net_norm0'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210808T154809_20210808T155521_T18SUJ_SERC_LAI_S2_20m_net_norm0'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190826T153819_20190826T154455_T18TYN_HOPB_LAI_S2_20m_net_norm0'}]

In [12]:
## make a dictionry list for reference image and the image (normed DSen2) to be compared based on site -new coefficents using thresholding based on bias (DSen2-normalized DSen2)
input_list4=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210719T190919_20210719T191700_T10TER_ABBY_LAI_DSen2_20m_net_norm1'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210401T183919_20210401T184709_T11SKB_SJER_LAI_DSen2_20m_net_norm1'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190822T173909_20190822T175453_T13SCS_JORN_LAI_DSen2_20m_net_norm1'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20200627T173909_20200627T174744_T14TLS_NOGP_LAI_DSen2_20m_net_norm1'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210422T162829_20210422T163638_T16SCA_LENO_LAI_DSen2_20m_net_norm1'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20200713T170901_20200713T171937_T14SQJ_MCDI_LAI_DSen2_20m_net_norm1'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190606T165901_20190606T170333_T16TCS_UNDE_LAI_DSen2_20m_net_norm1'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190608T164849_20190608T165019_T15TYL_STEI_LAI_DSen2_20m_net_norm1'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20210808T154809_20210808T155521_T18SUJ_SERC_LAI_DSen2_20m_net_norm1'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/DSen2_norm/20190826T153819_20190826T154455_T18TYN_HOPB_LAI_DSen2_20m_net_norm1'}]

In [13]:
## make a dictionry list for reference image and the image (normed S2)to be compared based on site -new coefficients -new coefficents using thresholding based on bias (S2-normalized S2)
input_list5=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210719T190919_20210719T191700_T10TER_ABBY_LAI_S2_20m_net_norm1'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210401T183919_20210401T184709_T11SKB_SJER_LAI_S2_20m_net_norm1'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190822T173909_20190822T175453_T13SCS_JORN_LAI_S2_20m_net_norm1'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20200627T173909_20200627T174744_T14TLS_NOGP_LAI_S2_20m_net_norm1'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210422T162829_20210422T163638_T16SCA_LENO_LAI_S2_20m_net_norm1'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20200713T170901_20200713T171937_T14SQJ_MCDI_LAI_S2_20m_net_norm1'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190606T165901_20190606T170333_T16TCS_UNDE_LAI_S2_20m_net_norm1'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190608T164849_20190608T165019_T15TYL_STEI_LAI_S2_20m_net_norm1'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20210808T154809_20210808T155521_T18SUJ_SERC_LAI_S2_20m_net_norm1'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/S2_norm/20190826T153819_20190826T154455_T18TYN_HOPB_LAI_S2_20m_net_norm1'}]

In [238]:
## make a dictionry list for reference image and the image (filter simulated NEON)to be compared based on site 
input_list6=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_10m_LAI_20m_net'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_10m_LAI_20m_net'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_10m_LAI_20m_net'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_10m_LAI_20m_net'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_10m_LAI_20m_net'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_10m_LAI_20m_net'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_10m_LAI_20m_net'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_10m_LAI_20m_net'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_10m_LAI_20m_net'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_10m_LAI_20m_net'}]

In [239]:
## make a dictionry list for reference image and the image (filter simulated NEON)to be compared based on site 
input_list7=[{'site':'ABBY','ref':'users/GangHong2/NEON_result/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D16_ABBY_DP1_20210719_191207_reflectance_filt_20m_LAI_20m_net'},
{'site':'SJER','ref':'users/GangHong2/NEON_result/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D17_SJER_DP1_20210331_200812_reflectance_filt_20m_LAI_20m_net'},
{'site':'JORN','ref':'users/GangHong2/NEON_result/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D14_JORN_DP1_20190825_165611_reflectance_filt_20m_LAI_20m_net'},
{'site':'NOGP','ref':'users/GangHong2/NEON_result/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D09_NOGP_DP1_20200626_152700_reflectance_filt_20m_LAI_20m_net'},
{'site':'LENO','ref':'users/GangHong2/NEON_result/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D08_LENO_DP1_20210422_172136_reflectance_filt_20m_LAI_20m_net'},
{'site':'MCDI','ref':'users/GangHong2/NEON_result/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D06_MCDI_DP1_20200713_192937_reflectance_filt_20m_LAI_20m_net'},
{'site':'UNDE','ref':'users/GangHong2/NEON_result/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D05_UNDE_DP1_20190606_184411_reflectance_filt_20m_LAI_20m_net'},
{'site':'STEI','ref':'users/GangHong2/NEON_result/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D05_STEI_DP1_20190608_194643_reflectance_filt_20m_LAI_20m_net'},
{'site':'SERC','ref':'users/GangHong2/NEON_result/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D02_SERC_DP1_20210811_142655_reflectance_filt_20m_LAI_20m_net'},
{'site':'HOPB','ref':'users/GangHong2/NEON_result/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m_LAI_20m_net','comp':'users/GangHong2/NEON_FILT_RESULT/NEON_D01_HOPB_DP1_20190826_172857_reflectance_filt_20m_LAI_20m_net'}]

In [248]:
def statsComp(inputlist):
        ## stats for LAI from filter simulated NEON 20m
    df_all = pd.DataFrame(columns=['site','RMSE','SSIM','R2','bias'])
    ## a loop for each site
    for input in inputlist:     
        site=input.get('site')
        # print (site)
        ref_img=ee.Image(input.get('ref'))
        comp_img=ee.Image(input.get('comp'))
        ## make a new image to combine biophycal parameter from differnent data inputs
        input_img=ref_img.select('estimateLAI').rename('NEON_10m_LAI').addBands(comp_img.select('estimateLAI').rename('DSen2_10m_LAI'))
        ## generate sample points 
        samples_feat = ee.FeatureCollection(input_img.sample(numPixels=1000, seed=1))
        ## get the propery name of the sample points
        samples_col=samples_feat.first().propertyNames().getInfo()
        ## remove property named 'system:index'
        samples_col.remove('system:index')
        ## get a new data frame for featuure list
        sample_list = samples_feat.reduceColumns(ee.Reducer.toList(len(samples_col)), samples_col).values().get(0)
        df=pd.DataFrame(sample_list.getInfo(), columns=samples_col) 
        #print (df.isnull().sum().sum()) ## check the sum of null in dataframe
        ## get the column value, need to be divided by 1000
        df2=df[(df > 0).all(1)]
        ref_Vals = df2[samples_col[0]]/1000
        predict_Vals = df2[samples_col[1]]/1000
        ## mean_squared_error
        # RMSE = mean_squared_error(ref_Vals, predict_Vals)  
        RMSE = mean_squared_error(ref_Vals, predict_Vals,squared=False)  
        # RMSE = np.sqrt(np.mean(np.power(ref_Vals-predict_Vals,2)))
        # print (mean_squared_error(ref_Vals, predict_Vals,squared=False)) 
        bias=np.mean(ref_Vals-predict_Vals)
        ## SSIM
        SSIM = ssim(ref_Vals, predict_Vals, data_range=(predict_Vals.max() - predict_Vals.min())) 
        # SSIM=0
        ## R square
        R2 = r2_score(ref_Vals, predict_Vals)
        # print (r2_score(ref_Vals, predict_Vals))
        # R2 =1-np.sum(np.power(ref_Vals-predict_Vals,2))/np.sum(np.power(ref_Vals,2))
        # R =1-np.sum(np.power(ref_Vals-predict_Vals,2))/np.sum(np.power(ref_Vals-np.mean(ref_Vals),2))
        # print (R)
        # df_temp={'site':site, 'RMSE': RMSE, 'SSIM':SSIM,'R2': R2}  
        # ## append the result of the site to the data frame
        # df_10m=df_10m.append(df_temp, ignore_index = True)

        df_temp = pd.DataFrame([{'site':site, 'RMSE': RMSE, 'SSIM':SSIM,'R2': R2, 'bias': bias}])
        # print (df_temp)
        df_all= pd.concat([df_all, df_temp], axis=0, ignore_index=True)
    
    print (df_all)

In [249]:
print ('*************************************************************************************************************************************')
print (' comparison between 10m simumated NEON and Gaussian Filtered 10m simulated NEON')
print ('*************************************************************************************************************************************')
statsComp(input_list6)

*************************************************************************************************************************************
 comparison between 10m simumated NEON and Gaussian Filtered 10m simulated NEON
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.108958  0.931896  0.931707 -0.003229
1  SJER  0.253687  0.871752  0.848846 -0.003431
2  JORN  0.064893  0.794540  0.804562  0.006580
3  NOGP  0.294818  0.918480  0.897137  0.025659
4  LENO  0.219478  0.945657  0.931485  0.002746
5  MCDI  0.203527  0.929046  0.901055 -0.003364
6  UNDE  0.155751  0.933916  0.930626  0.005013
7  STEI  0.182865  0.938517  0.925327  0.004194
8  SERC  0.169764  0.921586  0.895555 -0.011810
9  HOPB  0.121266  0.859400  0.856622 -0.001933


In [250]:
print ('*************************************************************************************************************************************')
print (' comparison between 10m simumated NEON and Gaussian Filtered 20m simulated NEON')
print ('*************************************************************************************************************************************')
statsComp(input_list7)

*************************************************************************************************************************************
 comparison between 10m simumated NEON and Gaussian Filtered 20m simulated NEON
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.139815  0.894433  0.884577 -0.001592
1  SJER  0.320170  0.806249  0.759241 -0.008502
2  JORN  0.091244  0.619296  0.613896  0.008253
3  NOGP  0.370100  0.873086  0.838359  0.029113
4  LENO  0.283722  0.910516  0.885505 -0.004078
5  MCDI  0.265067  0.877858  0.831356 -0.002616
6  UNDE  0.191520  0.902758  0.895102  0.008997
7  STEI  0.232539  0.895050  0.879247  0.010062
8  SERC  0.221523  0.866004  0.822159 -0.021647
9  HOPB  0.163165  0.781795  0.740425 -0.002568


In [251]:
print ('*************************************************************************************************************************************')
print (' comparison between 10m simumated NEON and nomalized DSen2')
print ('*************************************************************************************************************************************')
statsComp(input_list2)

*************************************************************************************************************************************
 comparison between 10m simumated NEON and nomalized DSen2
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.230528  0.757686  0.679400 -0.045519
1  SJER  0.331125  0.804405  0.739368 -0.095282
2  JORN  0.108756  0.551250  0.451466  0.050297
3  NOGP  0.484436  0.841621  0.722571  0.286644
4  LENO  0.356004  0.775784  0.694776 -0.072078
5  MCDI  0.256978  0.890203  0.842260 -0.045110
6  UNDE  0.223313  0.840314  0.792031  0.000740
7  STEI  0.329660  0.824679  0.751234 -0.154060
8  SERC  0.768967  0.670907 -1.180367  0.715020
9  HOPB  0.222747  0.680717  0.516238  0.106727


In [252]:
print ('*************************************************************************************************************************************')
print (' comparison between 10m simumated NEON and normalized S2')
print ('*************************************************************************************************************************************')
statsComp(input_list3)

*************************************************************************************************************************************
 comparison between 10m simumated NEON and normalized S2
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.214604  0.785732  0.722164 -0.042281
1  SJER  0.348893  0.774756  0.710648 -0.086584
2  JORN  0.107467  0.568219  0.464392  0.048211
3  NOGP  0.511592  0.818188  0.690595  0.292275
4  LENO  0.349751  0.776751  0.705405 -0.071209
5  MCDI  0.300236  0.845984  0.784684 -0.043895
6  UNDE  0.244103  0.811674  0.751504  0.002429
7  STEI  0.321524  0.821959  0.763361 -0.135199
8  SERC  0.777454  0.655818 -1.228761  0.719350
9  HOPB  0.223997  0.662405  0.510796  0.107376


In [253]:
print ('*************************************************************************************************************************************')
print (' comparison between 10m simumated NEON and nomalized DSen2 using thresholding')
print ('*************************************************************************************************************************************')
statsComp(input_list4)

*************************************************************************************************************************************
 comparison between 10m simumated NEON and nomalized DSen2 using thresholding
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.230013  0.758069  0.680832 -0.043412
1  SJER  0.335821  0.803998  0.731924 -0.110012
2  JORN  0.109492  0.550050  0.444024  0.051833
3  NOGP  0.498019  0.838910  0.706794  0.308944
4  LENO  0.354625  0.775262  0.697137 -0.066256
5  MCDI  0.256740  0.890296  0.842552 -0.044133
6  UNDE  0.223701  0.840223  0.791307 -0.007858
7  STEI  0.330959  0.824482  0.749270 -0.156793
8  SERC  0.835442  0.645452 -1.573635  0.784148
9  HOPB  0.220555  0.679404  0.525713  0.101982


In [254]:
print ('*************************************************************************************************************************************')
print (' comparison between 10m simumated NEON and nomalized S2 using thresholding')
print ('*************************************************************************************************************************************')
statsComp(input_list5)

*************************************************************************************************************************************
 comparison between 10m simumated NEON and nomalized S2 using thresholding
*************************************************************************************************************************************
   site      RMSE      SSIM        R2      bias
0  ABBY  0.214064  0.786086  0.723558 -0.040207
1  SJER  0.352912  0.774479  0.703942 -0.101301
2  JORN  0.108226  0.562824  0.456515  0.049715
3  NOGP  0.524734  0.815509  0.674495  0.314596
4  LENO  0.348889  0.775775  0.706855 -0.065435
5  MCDI  0.300093  0.846026  0.784889 -0.042916
6  UNDE  0.244343  0.811683  0.751017 -0.006190
7  STEI  0.322819  0.821676  0.761451 -0.137936
8  SERC  0.843577  0.629501 -1.624002  0.788218
9  HOPB  0.222071  0.660896  0.519173  0.102627


## Aggregate to 30m ( 3 by 3 pixel Aggregated from 10m data, DSen2)

In [ ]:
# ## Stats for LAI from DSen2 without normalization
# df_30m = pd.DataFrame(columns=['site','RMSE','SSIM','R2'])
# ## a loop for each site
# for input in input_list2:
#     site=input.get('site')
#     ## get the orignal 10m data from reference image and arregate to 30m
#     ref_img=ee.Image(input.get('ref'))
#     proj_30m_ref= ref_img.projection().scale(3, 3) 
#     # print (proj_30m_ref.getInfo())
#     ref_img_30m=ref_img.select('estimateLAI').reduceResolution(
#         **{
#           'reducer': ee.Reducer.mean(),
#           'maxPixels': 1112
#         }) \
#         .reproject(
#             **{
#           'crs': proj_30m_ref
#         }).rename('NEON_30m_LAI')
#      ## get the orignal 10m data from the image to be compared and arregate to 30m
#     comp_img=ee.Image(input.get('comp'))  
#     proj_30m_comp=comp_img.projection().scale(3, 3) 
#     comp_img_30m=comp_img.select('estimateLAI').reduceResolution(
#         **{
#           'reducer': ee.Reducer.mean(),
#           'maxPixels': 1112
#         }) \
#         .reproject(
#             **{
#           'crs': proj_30m_comp
#         }).rename('DSen2_30m_LAI')
#    ## genreate a new image to combine 30m reference image and the image to be compared
#     input_img_30m=ref_img_30m.select('NEON_30m_LAI').addBands(comp_img_30m.select('DSen2_30m_LAI'))
#     ## generate samples
#     samples_feat_30m = ee.FeatureCollection(input_img_30m.sample(numPixels=1000, seed=1))
#     ## get the property name of sample features
#     samples_col_30m=samples_feat_30m.first().propertyNames().getInfo()##.remove('system:index')
#     ## remove the property 'system:index'
#     samples_col_30m.remove('system:index')
#     # print (samples_col_30m)
#     ## convert the data from ee.feature to data frame
#     sample_list_30m = samples_feat_30m.reduceColumns(ee.Reducer.toList(len(samples_col_30m)), samples_col_30m).values().get(0)
#     df2=pd.DataFrame(sample_list_30m.getInfo(), columns=samples_col_30m) 
#     df[(df > 0).all(1)]
#     # print (df2.isnull().sum().sum()) ## check the sum of null in dataframe
#     # print (df2.head(10))
#     ## get the value for each column
#     ref_Vals = df2[samples_col_30m[0]]/1000
#     predict_Vals = df2[samples_col_30m[1]]/1000
#     ## RMSE
#     RMSE = mean_squared_error(ref_Vals, predict_Vals) 
#     ## ssim
#     SSIM = ssim(ref_Vals, predict_Vals, data_range=(predict_Vals.max() - predict_Vals.min())) 
#     ## R2
#     R2 = r2_score(ref_Vals, predict_Vals)
#     # df_temp={'site':site, 'RMSE': RMSE, 'SSIM':SSIM,'R2': R2}   
#     # ## append the result of the site to the data frame
#     # df_30m=df_30m.append(df_temp, ignore_index = True)
#     df_temp = pd.DataFrame([{'site':site, 'RMSE': RMSE, 'SSIM':SSIM,'R2': R2}])
#     df_30m= pd.concat([df_30m, df_temp], axis=0, ignore_index=True)
# print (df_30m)